## 아이리스 다중 분류

In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import pandas as pd
df = pd.read_csv('dataset/iris.csv', 
                 names = ["sepal_length", "sepal_width", "petal_length", "petal_width", "species"])
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
dataset = df.values
x_data = np.array(dataset[:,0:4], dtype=np.float32)
Y_obj = dataset[:,4]
Y_obj[0], Y_obj[50], Y_obj[100]

('Iris-setosa', 'Iris-versicolor', 'Iris-virginica')

In [4]:
Y_obj = pd.get_dummies(Y_obj)
Y_obj.head()

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [5]:
type(Y_obj), type(x_data)

(pandas.core.frame.DataFrame, numpy.ndarray)

In [6]:
y_data = np.array(Y_obj.values, dtype=np.float32)
y_data[0], y_data[50], y_data[100]

(array([1., 0., 0.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 0., 1.], dtype=float32))

In [7]:
print(x_data.shape, y_data.shape)

(150, 4) (150, 3)


In [8]:
# seed 값 설정
seed = 0
tf.set_random_seed(seed)

In [9]:
# 입력 값을 플레이스 홀더에 저장
X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 3])

In [10]:
# Layer 1
W1 = tf.Variable(tf.random_normal([4,16], dtype=tf.float32)) 
b1 = tf.Variable(tf.random_normal([16], dtype=tf.float32))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

In [11]:
# Layer 2
W2 = tf.Variable(tf.random_normal([16,1], dtype=tf.float32)) 
b2 = tf.Variable(tf.random_normal([1], dtype=tf.float32))
#logits = tf.matmul(L1, W2) + b2
hypo = tf.nn.softmax(tf.matmul(L1, W2) + b2)

In [12]:
# cross-entropy
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_data))
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypo)))

In [15]:
learning_rate = 0.1
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)
#train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [16]:
# 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(51):
    sess.run(train, feed_dict = {X: x_data, Y: y_data})
    if epoch % 10 == 0:
        cost_val = sess.run(cost, feed_dict = {X: x_data, Y: y_data})
        print("step=%d, \tcost_val=%.5f" % (epoch + 1, cost_val))

step=1, 	cost_val=-0.00000
step=11, 	cost_val=-0.00000
step=21, 	cost_val=-0.00000
step=31, 	cost_val=-0.00000
step=41, 	cost_val=-0.00000
step=51, 	cost_val=-0.00000


In [17]:
predict = tf.equal(tf.argmax(y_data, 1), tf.argmax(hypo, 1))
accuracy = tf.reduce_mean(tf.cast(predict, dtype=tf.float64))

In [18]:
acc = sess.run(accuracy, feed_dict = {X: x_data, Y: y_data})
print("Accuracy: %.4f" % acc)

Accuracy: 0.3333


In [13]:
# 딥러닝 모델 설정
model = Sequential([
    Dense(16, input_shape=(4,), activation='relu'),
    Dense(3, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                80        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 51        
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


In [14]:
# 모델 컴파일 
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

In [18]:
# 모델 실행
model.fit(X, Y_encoded, epochs=50, batch_size=10)

Train on 150 samples
Epoch 1/50
150/150 [==============================] - 0s 246us/sample - loss: 0.2945 - accuracy: 0.9667
Epoch 2/50
150/150 [==============================] - 0s 220us/sample - loss: 0.2896 - accuracy: 0.9800
Epoch 3/50
150/150 [==============================] - 0s 246us/sample - loss: 0.2861 - accuracy: 0.9733
Epoch 4/50
150/150 [==============================] - 0s 246us/sample - loss: 0.2816 - accuracy: 0.9733
Epoch 5/50
150/150 [==============================] - 0s 246us/sample - loss: 0.2799 - accuracy: 0.9800
Epoch 6/50
150/150 [==============================] - 0s 239us/sample - loss: 0.2748 - accuracy: 0.9667
Epoch 7/50
150/150 [==============================] - 0s 247us/sample - loss: 0.2726 - accuracy: 0.9800
Epoch 8/50
150/150 [==============================] - 0s 246us/sample - loss: 0.2655 - accuracy: 0.9800
Epoch 9/50
150/150 [==============================] - 0s 246us/sample - loss: 0.2635 - accuracy: 0.9733
Epoch 10/50
150/150 [======================

In [19]:
# 결과 출력 
print("\n Accuracy: %.4f" % (model.evaluate(X, Y_encoded, verbose=0)[1]))


 Accuracy: 0.9733
